In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
%matplotlib inline


### Eliminare colonne non necessarie e righe che contengono _Current Ver_ == _Varies with device_

In [2]:
df = pd.read_csv('/Users/andreafavia/Desktop/google-play-store-apps/googleplaystore.csv')
df.head(50)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


In [3]:
df = pd.read_csv('/Users/andreafavia/Desktop/google-play-store-apps/googleplaystore.csv')
df = df.drop(['Type','App','Content Rating','Genres','Last Updated','Android Ver'],axis=1)
df = df[df['Current Ver'] != 'Varies with device']
df = df[~df['Current Ver'].isnull()]
df = df[df['Size'] != 'Varies with device']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9065 entries, 0 to 10838
Data columns (total 7 columns):
Category       9065 non-null object
Rating         7653 non-null float64
Reviews        9065 non-null object
Size           9065 non-null object
Installs       9065 non-null object
Price          9065 non-null object
Current Ver    9065 non-null object
dtypes: float64(1), object(6)
memory usage: 566.6+ KB


## Data Cleaning
1. NaN values in _Rating_ sostituiti con la media dei non numeri non nulli

In [4]:
mean = df.Rating.mean()
df.Rating = df.Rating.fillna(round(mean,1))

2. _Size_ espresso con float32 ( elimino M e k )

In [5]:
idx_to_drop = df[ df.Size.str.contains(',')].index[0]
df = df.drop(idx_to_drop)

In [6]:
def transform_size(x):
    p = 0;
    if x[-1] == 'M':
        return float(x[:-1])
    return (float(x[:-1])/1000)

In [7]:
df.Size = df.Size.apply(transform_size)
df.Size.dtype

dtype('float64')

3. _Install_ espresso con int32 ( elimino '+' e ',' per parsing numerico)


In [8]:
df.Installs = df.Installs.apply(lambda x: x.replace(',',''))
df.Installs = df.Installs.apply(lambda x: int(x.replace('+','')))

4. _Price_ espresso con float32 ( elimino \\$ per parsing numerico)

In [9]:
df.Price = df.Price.apply(lambda x: x.replace('$',''))
#df[ df.Price != '0']
df.Price = df.Price.apply(lambda x: float(x))

In [10]:
df = df.rename(columns = {'Size':'Size_Mega','Price':'Price_dollar'})

5. Transform _Current Ver_ in format { xx.yy }

In [11]:
def transform_version(x):
    pat = r'(?P<main>\d+)(\.(?P<second>\d+)(\.(\d+))?)?'
    m = re.compile(pat)
    if m.search(x) is not None:
        v = m.search(x)
        return v.group('main') +'.'+ v.group('second') if  v.group('second') is not None else v.group('main')
    else:
        return 'None'

In [12]:
df['Current Ver'] = df['Current Ver'].apply(transform_version)
df = df[df['Current Ver'] != 'None']

In [13]:
df.sample(20)

,Category,Rating,Reviews,Size_Mega,Installs,Price_dollar,Current Ver
2317,MEDICAL,2.7,2657,24.0,500000,0.00,8.0
2380,MEDICAL,4.5,171,20.0,10000,10.00,1.8
6559,FAMILY,4.2,2,8.5,10,46.99,1.0
8059,FAMILY,3.0,2,33.0,100,0.00,3.0
7004,FAMILY,4.4,23474,8.8,5000000,0.00,1.0
444,COMMUNICATION,4.4,93825,11.0,5000000,0.00,2.17
1701,GAME,4.5,4447346,67.0,100000000,0.00,2.347
5295,TOOLS,3.9,54,2.8,10000,0.00,1.0
8089,NEWS_AND_MAGAZINES,4.5,27,7.1,500,0.00,7.5
492,DATING,3.1,31320,53.0,5000000,0.00,6.6


## Description of some data

In [14]:
df.describe()

,Rating,Size_Mega,Installs,Price_dollar
count,9047.000000,9047.000000,9.047000e+03,9047.000000
mean,4.176733,21.488903,6.902186e+06,1.148848
std,0.501837,22.583668,4.549337e+07,16.937144
min,1.000000,0.008500,0.000000e+00,0.000000
25%,4.000000,4.900000,1.000000e+03,0.000000
50%,4.200000,13.000000,1.000000e+05,0.000000
75%,4.500000,30.000000,1.000000e+06,0.000000
max,5.000000,100.000000,1.000000e+09,400.000000


## Save dataset to csv


In [ ]:
df.to_csv('google_play_store_cleaned.csv',index = False)

In [ ]:
dc = pd.read_csv('google_play_store_cleaned.csv')